In [2]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

# 1. Download and load the specific geo dataset
import io
import requests
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = requests.get(URL)
spacex_csv_file = io.BytesIO(resp.content)
spacex_df = pd.read_csv(spacex_csv_file)

# 2. Select relevant sub-columns as the lab instructs
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

In [3]:
# Initialize the map at NASA Johnson Space Center
nasa_coordinate = [29.5596848, -95.0830971]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# The lab instruction: Create a blue circle at NASA with a popup label
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [4]:
marker_cluster = MarkerCluster()

# Function to fulfill the lab's "marker_color" task
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Add cluster to map
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    # Create a Marker object with its coordinate and customize the icon
    marker = folium.Marker([record['Lat'], record['Long']],
                           icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

site_map

In [5]:
# Add Mouse Position to get coordinates for distance calculation
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

In [6]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [7]:
# 1. Define the coordinates
launch_site_lat = 28.562302
launch_site_lon = -80.577356
coastline_lat = 28.56367
coastline_lon = -80.56763

# 2. Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# 3. Create a marker for the distance
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
site_map.add_child(distance_marker)

# 4. Create and add the PolyLine
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)
site_map.add_child(lines)

site_map

In [8]:
# List of points to map [lat, lon, label]
proximities = [
    [28.56335, -80.57085, 'Highway'],
    [28.57206, -80.58525, 'Railway'],
    [28.61200, -80.80700, 'City']
]

for point in proximities:
    # 1. Calculate distance
    dist = calculate_distance(launch_site_lat, launch_site_lon, point[0], point[1])

    # 2. Add Distance Marker
    distance_marker = folium.Marker(
        [point[0], point[1]],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(dist),
        )
    )
    site_map.add_child(distance_marker)

    # 3. Add PolyLine
    line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [point[0], point[1]]], weight=1)
    site_map.add_child(line)

site_map